In [1]:
!pip install dagshub mlflow --quiet

import warnings
warnings.filterwarnings("ignore")

print("Done!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 2.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 57.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 40.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.5/95.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv
/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip
/kaggle/input/walmart-recruiting-store-sales-forecasting/test.csv.zip


In [3]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import mlflow
import mlflow.sklearn
import dagshub
import joblib
import os

dagshub.init(repo_owner='gnada22', repo_name='ml_final_project', mlflow=True)

mlflow.set_experiment("Linear_Regression_Training")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=c38fa121-750c-4b00-8f4f-9ab6c8c35c93&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=29f7b03b8981d12aa23bc404e41f795ad521eba8be2833d74cf18cd23ce3aca1




Output()

Accessing as gnada22

Initialized MLflow to track repo "gnada22/ml_final_project"

Repository gnada22/ml_final_project initialized!

<Experiment: artifact_location='mlflow-artifacts:/bdaee488c065473c96d054e5ae43539a', creation_time=1751293431320, experiment_id='0', last_update_time=1751293431320, lifecycle_stage='active', name='Linear_Regression_Training', tags={}>

In [6]:
class DateFeatureCreator(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X["weekofyear"] = X["Date"].dt.isocalendar().week.astype(int)
        X["sin_13"] = np.sin(2 * np.pi * X["weekofyear"] / 13)
        X["cos_13"] = np.cos(2 * np.pi * X["weekofyear"] / 13)
        X["sin_23"] = np.sin(2 * np.pi * X["weekofyear"] / 23)
        X["cos_23"] = np.cos(2 * np.pi * X["weekofyear"] / 23)
        return X

date_features = ["weekofyear", "sin_13", "cos_13", "sin_23", "cos_23"]

class LagFeatureAdder:
    def transform(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.sort_values(["Store", "Dept", "Date"])
        df["lag_1"] = df.groupby(["Store", "Dept"])["Weekly_Sales"].shift(1)
        df["lag_52"] = df.groupby(["Store", "Dept"])["Weekly_Sales"].shift(52)
        return df

lag_features = ["lag_1", "lag_52"]

added_features = date_features + lag_features

In [7]:
# Load and merge data
df_train = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/train.csv.zip", parse_dates=["Date"])
df_features = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/features.csv.zip", parse_dates=["Date"])
df_stores = pd.read_csv("/kaggle/input/walmart-recruiting-store-sales-forecasting/stores.csv")

df = df_train.merge(df_features, on=["Store", "Date", "IsHoliday"], how="left")
df = df.merge(df_stores, on="Store", how="left")

In [10]:
categorical = ["Store", "Dept", "Type"]
numerical = ["Temperature", "Fuel_Price", "CPI", "Unemployment",
             "MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5"]
engineered = added_features

In [11]:
categorical_transformer = OneHotEncoder(handle_unknown="ignore")
numerical_transformer = SimpleImputer(strategy="mean")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical + engineered),
        ("cat", categorical_transformer, categorical),
    ]
)

pipeline = Pipeline([
    ("date_features", DateFeatureCreator()),
    ("preprocessor", preprocessor),
    ("regressor", LinearRegression())
])

In [12]:
print(df.columns)

with mlflow.start_run(run_name="feature_engineering"):
    lag_adder = LagFeatureAdder()
    df = lag_adder.transform(df)
    original_rows = len(df)
    df = df.dropna(subset=["lag_1", "lag_52"])
    remaining_rows = len(df)
    mlflow.log_param("rows_before_dropna", original_rows)
    mlflow.log_param("rows_after_dropna", remaining_rows)
    mlflow.log_param("features_added", added_features)

y = df["Weekly_Sales"]
X = df.drop(columns=["Weekly_Sales"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday', 'Temperature',
       'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4',
       'MarkDown5', 'CPI', 'Unemployment', 'Type', 'Size'],
      dtype='object')
🏃 View run feature_engineering at: https://dagshub.com/gnada22/ml_final_project.mlflow/#/experiments/0/runs/d8fe86d5ad264fd48fccc4848b906523
🧪 View experiment at: https://dagshub.com/gnada22/ml_final_project.mlflow/#/experiments/0


In [13]:
print(X_train.columns)

with mlflow.start_run(run_name="training"):
    pipeline.fit(X_train, y_train)
    preds = pipeline.predict(X_val)

    mae = mean_absolute_error(y_val, preds)
    weights = X_val["IsHoliday"].apply(lambda x: 5 if x else 1)
    wmae = (weights * np.abs(y_val - preds)).sum() / weights.sum()

    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("WMAE", wmae)
    # mlflow.sklearn.log_model(pipeline, artifact_path="model")

    model_path = "model.pkl"
    joblib.dump(pipeline, model_path)
    mlflow.log_artifact(model_path)

Index(['Store', 'Dept', 'Date', 'IsHoliday', 'Temperature', 'Fuel_Price',
       'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI',
       'Unemployment', 'Type', 'Size', 'lag_1', 'lag_52'],
      dtype='object')
🏃 View run training at: https://dagshub.com/gnada22/ml_final_project.mlflow/#/experiments/0/runs/3db5b8f232c4413899af75467065c3bd
🧪 View experiment at: https://dagshub.com/gnada22/ml_final_project.mlflow/#/experiments/0
